# 순환 신경망(Recurrent Neural Network)

앞서 배운 피드 포워드 신경망은 입력의 길이가 고정되어 있어 자연어 처리를 위한 신경망으로는 한계가 있었습니다. 결국 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망이 필요하게 되었는데, 자연어 처리에 대표적으로 사용되는 인공 신경망인 RNN, LSTM 등에 대해서 알아봅니다.

## 1) 순환 신경망(Recurrent Neural Network, RNN)

RNN(Recurrent Neural Network)은 시퀀스(Sequence) 모델입니다. 입력과 출력을 시퀀스 단위로 처리하는 모델입니다. 번역기를 생각해보면 입력은 번역하고자 하는 문장. 즉, 단어 시퀀스입니다. 출력에 해당되는 번역된 문장 또한 단어 시퀀스입니다. 이러한 시퀀스들을 처리하기 위해 고안된 모델들을 시퀀스 모델이라고 합니다. 그 중에서도 RNN은 딥 러닝에 있어 가장 기본적인 시퀀스 모델입니다.

### 1. 순환 신경망 구현하기

케라스로 RNN 층을 추가하는 코드는 다음과 같습니다.

In [ ]:
# RNN 층을 추가하는 코드.
model.add(SimpleRNN(hidden_size)) # 가장 간단한 형태

인자를 사용할 때를 보겠습니다.

In [ ]:
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N))
# 단, M과 N은 정수

hidden_size = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일.   
RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가진다.  
timesteps = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수.  
input_dim = 입력의 크기.

![](img/rnn_image6between7.png)

In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


출력값이 (batch_size, output_dim) 크기의 2D 텐서일 때, output_dim은 hidden_size의 값인 3입니다. 이 경우 batch_size를 현 단계에서는 알 수 없으므로 (None, 3)이 됩니다. 이번에는 batch_size를 미리 정의해보겠습니다.

In [3]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


batch_size를 8로 기재하자, 출력의 크기가 (8, 3)이 된 것을 볼 수 있습니다. 이제 return_sequences 매개 변수에 True를 기재하여 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴하도록 모델을 만들어 보도록 하겠습니다.

In [4]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


출력의 크기가 (8, 2, 3)이 된 것을 확인할 수 있습니다.

## 2) RNN을 이용한 텍스트 생성(Text Generation using RNN)

이번 챕터에서는 다 대 일(many-to-one) 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델을 만들어봅시다.

### 1. RNN을 이용하여 텍스트 생성하기

예를 들어서 '경마장에 있는 말이 뛰고 있다'와 '그의 말이 법이다'와 '가는 말이 고와야 오는 말이 곱다'라는 세 가지 문장이 있다고 해봅시다. 모델이 문맥을 학습할 수 있도록 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 재구성한다면 아래와 같이 총 11개의 샘플이 구성됩니다.


|samples|X|y|
|-----------|---|---|
|1|경마장에|있는|
|2|경마장에 있는|말이|
|3|경마장에 있는 말이|뛰고|
|4|경마장에 있는 말이 뛰고|있다|
|5|그의|말이|
|6|그의 말이|법이다|
|7|가는|말이|
|8|가는 말이|고와야|
|9|가는 말이 고와야|오는|
|10|가는 말이 고와야 오는|말이|
|11|가는 말이 고와야 오는 말이|곱다|

#### 1) 데이터에 대한 이해와 전처리

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

우선 예제로 언급한 3개의 한국어 문장을 저장합니다.

In [6]:
text="""경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

단어 집합을 생성하고 크기를 확인해보겠습니다.

In [8]:
t = Tokenizer()
print(text)
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
단어 집합의 크기 : 12


각 단어와 단어에 부여된 정수 인덱스를 출력해보겠습니다.

In [10]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


이제 훈련 데이터를 만들어보겠습니다.

In [13]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    print(line)
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))


경마장에 있는 말이 뛰고 있다
[2, 3, 1, 4, 5]
그의 말이 법이다
[6, 1, 7]
가는 말이 고와야 오는 말이 곱다
[8, 1, 9, 10, 1, 11]
학습에 사용할 샘플의 개수: 11


샘플의 개수는 총 11개가 나옵니다. 전체 샘플을 출력해봅시다.

In [14]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다. [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준으로 합니다. 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다. 이를 코드로는 다음과 같이 구할 수 있습니다.

In [15]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인하였습니다. 이제 전체 샘플의 길이를 6으로 패딩합니다.

In [16]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

pad_sequences()는 모든 샘플에 대해서 0을 사용하여 길이를 맞춰줍니다. maxlen의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰주며, padding의 인자로 'pre'를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채웁니다. 전체 훈련 데이터를 출력해봅니다.

In [17]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


길이가 6보다 짧은 모든 샘플에 대해서 앞에 0을 채워서 모든 샘플의 길이를 6으로 바꿨습니다. 이제 각 샘플의 마지막 단어를 레이블로 분리합시다. 레이블의 분리는 Numpy를 이용해서 가능합니다.

In [26]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

분리된 X와 y에 대해서 출력해보면 다음과 같습니다.

In [27]:
print(X)
print(X.shape)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
(11, 5)


In [28]:
print(y) # 모든 샘플에 대한 레이블 출력
print(y.shape)

[ 3  1  4  5  1  7  1  9 10  1 11]
(11,)


레이블이 분리되었습니다. 이제 RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행합니다.

In [29]:
y = to_categorical(y, num_classes=vocab_size)

원-핫 인코딩이 수행되었는지 출력합니다.

In [30]:
print(y)
print(y.shape)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
(11, 12)


정상적으로 원-핫 인코딩이 수행된 것을 볼 수 있습니다.

#### 2) 모델 설계하기

이제 RNN 모델에 데이터를 훈련시킵니다.

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.5028 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4868 - accuracy: 0.0000e+00
Epoch 3/200
1/1 - 0s - loss: 2.4714 - accuracy: 0.0000e+00
Epoch 4/200
1/1 - 0s - loss: 2.4565 - accuracy: 0.0909
Epoch 5/200
1/1 - 0s - loss: 2.4417 - accuracy: 0.0909
Epoch 6/200
1/1 - 0s - loss: 2.4270 - accuracy: 0.4545
Epoch 7/200
1/1 - 0s - loss: 2.4122 - accuracy: 0.4545
Epoch 8/200
1/1 - 0s - loss: 2.3970 - accuracy: 0.4545
Epoch 9/200
1/1 - 0s - loss: 2.3814 - accuracy: 0.3636
Epoch 10/200
1/1 - 0s - loss: 2.3653 - accuracy: 0.3636
Epoch 11/200
1/1 - 0s - loss: 2.3485 - accuracy: 0.3636
Epoch 12/200
1/1 - 0s - loss: 2.3310 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.3127 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.2935 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.2735 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2526 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.2309 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.2082 - accuracy: 

Epoch 147/200
1/1 - 0s - loss: 0.2364 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.2319 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.2273 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.2229 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.2186 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.2143 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.2101 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.2060 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.2020 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.1980 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.1942 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1904 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1866 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1830 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1794 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1759 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1725 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

각 단어의 임베딩 벡터는 10차원을 가지고, 32의 은닉 상태 크기를 가지는 바닐라 RNN을 사용합니다.  
모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어서 출력해봅시다.

In [32]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

이제 입력된 단어로부터 다음 단어를 예측해서 문장을 생성하는 함수를 만들었습니다.

In [33]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

C:\Users\jikim\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [34]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [35]:
print(sentence_generation(model, t, '가는', 3)) # 5번 예측

가는 말이 고와야 오는


이제 앞의 문맥을 기준으로 '말이' 라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여줍니다. 이 모델은 충분한 훈련 데이터를 갖고 있지 못하므로 위에서 문장의 길이에 맞게 적절하게 예측해야하는 횟수 4, 2, 5를 각각 인자값으로 주었습니다. 이 이상의 숫자를 주면 기계는 '있다', '법이다', '곱다' 다음에 나오는 단어가 무엇인지 배운 적이 없으므로 임의 예측을 합니다. 이번에는 더 많은 훈련 데이터를 가지고 실습해봅시다.

### 2. LSTM을 이용하여 텍스트 생성하기

이번에는 LSTM을 통해 보다 많은 데이터로 텍스트를 생성해보겠습니다. 본질적으로 앞에서 한 것과 동일한 실습입니다.

#### 1) 데이터에 대한 이해와 전처리

사용할 데이터는 뉴욕 타임즈 기사의 제목입니다. 아래의 링크에서 ArticlesApril2018.csv 데이터를 다운로드 합니다.

파일 다운로드 링크 : https://www.kaggle.com/aashita/nyt-comments

In [37]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

다운로드한 훈련 데이터를 데이터프레임에 저장합니다.

In [39]:
df=pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


열의 개수가 굉장히 많기 때문에 한 눈에 보기 어렵습니다. 어떤 열이 있고, 열이 총 몇 개가 있는지 출력해봅시다.

In [40]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


총 15개의 열이 존재합니다. 여기서 사용할 열은 제목에 해당되는 headline 열입니다. Null 값이 있는지 확인해봅시다.

In [41]:
df['headline'].isnull().values.any()

False

Null 값은 별도로 없는 것으로 보입니다. headline 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장해보도록 하겠습니다.

In [43]:
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

headline이라는 리스트에 모든 신문 기사의 제목을 저장했습니다. 저장한 리스트에서 상위 5개만 출력해보았습니다.

그런데 4번째, 5번째 샘플에 Unknown 값이 들어가있습니다. headline 전체에 걸쳐서 Unknown 값을 가진 샘플이 있을 것으로 추정됩니다. 비록 Null 값은 아니지만 지금 하고자 하는 실습에 도움이 되지 않는 노이즈 데이터이므로 제거해줄 필요가 있습니다. 제거하기 전에 현재 샘플의 개수를 확인해보고 제거 전, 후의 샘플의 개수를 비교해보겠습니다.

In [44]:
print('총 샘플의 개수 : {0}'.format(len(headline))) # 현재 샘플의 개수

총 샘플의 개수 : 1324


노이즈 데이터를 제거하기 전 데이터의 개수는 1,324입니다. 즉, 신문 기사의 제목이 총 1,324개입니다.

In [45]:
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

노이즈값 제거 후 샘플의 개수 : 1214


샘플의 수가 1,324에서 1,214로 110개의 샘플이 제거되었는데, 기존에 출력했던 5개의 샘플을 출력해보겠습니다.

In [46]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

기존에 4번째, 5번째 샘플에서는 Unknown 값이 있었는데 현재는 제거가 된 것을 확인하였습니다. 이제 데이터 전처리를 수행합니다. 여기서 선택한 전처리는 구두점 제거와 단어의 소문자화입니다. 전처리를 수행하고, 다시 샘플 5개를 출력합니다.

In [47]:
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

기존의 출력과 비교하면 모든 단어들이 소문자화되었으며 N.F.L.이나 Cheerleaders’ 등과 같이 기존에 구두점이 붙어있던 단어들에서 구두점이 제거되었습니다. 이제 단어 집합(vocabulary)을 만들고 크기를 확인합니다.

In [49]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


총 3,494개의 단어가 존재합니다. 이제 정수 인코딩과 동시에 하나의 문장을 여러 줄로 분해하여 훈련 데이터를 구성합니다.

In [50]:
sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

이해를 돕기 위해 출력 결과에 주석을 추가하였습니다. 왜 하나의 문장을 저렇게 나눌까요? 예를 들어 '경마장에 있는 말이 뛰고 있다' 라는 문장 하나가 있을 때, 최종적으로 원하는 훈련 데이터의 형태는 다음과 같습니다. 하나의 단어를 예측하기 위해 이전에 등장한 단어들을 모두 참고하는 것입니다.

|samples|X|y|
|-----------|---|---|
|1|경마장에|있는|
|2|경마장에 있는|말이|
|3|경마장에 있는 말이|뛰고|
|4|경마장에 있는 말이 뛰고|있다|

위의 sequences는 모든 문장을 각 단어가 각 시점(time step)마다 하나씩 추가적으로 등장하는 형태로 만들기는 했지만, 아직 예측할 단어에 해당되는 레이블을 분리하는 작업까지는 수행하지 않은 상태입니다. 어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만듭니다.

In [51]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


582이라는 인덱스를 가진 단어는 본래 offer라는 단어였습니다. 원한다면 다른 숫자로도 시도해보세요. 이제 y데이터를 분리하기 전에 전체 샘플의 길이를 동일하게 만드는 패딩 작업을 수행합니다. 패딩 작업을 수행하기 전에 가장 긴 샘플의 길이를 확인합니다.

In [52]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


가장 긴 샘플의 길이인 24로 모든 샘플의 길이를 패딩하겠습니다.

In [53]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


padding='pre'를 설정하여 샘플의 길이가 24보다 짧은 경우에 앞에 0으로 패딩되었습니다. 이제 맨 우측 단어만 레이블로 분리합니다.

In [69]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [70]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


훈련 데이터 X에서 3개의 샘플만 출력해보았는데, 맨 우측에 있던 정수값 269, 371, 1115가 사라진 것을 볼 수 있습니다. 뿐만 아니라, 각 샘플의 길이가 24에서 23으로 줄었습니다.

In [71]:
print(y[:3]) # 레이블

[ 269  371 1115]


훈련 데이터 y 중 3개의 샘플만 출력해보았는데, 기존 훈련 데이터에서 맨 우측에 있던 정수들이 별도로 저장되었습니다.

In [74]:
y_onehot = to_categorical(y, num_classes=vocab_size)
y_onehot.shape

(7803, 3494)

레이블 데이터 y에 대해서 원-핫 인코딩을 수행하였습니다. 이제 모델을 설계합니다.

#### 2) 모델 설계하기

In [76]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [77]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y_onehot, epochs=200, verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.6556 - accuracy: 0.0273
Epoch 2/200
244/244 - 2s - loss: 7.1139 - accuracy: 0.0299
Epoch 3/200
244/244 - 2s - loss: 6.9741 - accuracy: 0.0363
Epoch 4/200
244/244 - 2s - loss: 6.8469 - accuracy: 0.0414
Epoch 5/200
244/244 - 3s - loss: 6.6995 - accuracy: 0.0433
Epoch 6/200
244/244 - 3s - loss: 6.5377 - accuracy: 0.0484
Epoch 7/200
244/244 - 2s - loss: 6.3640 - accuracy: 0.0505
Epoch 8/200
244/244 - 1s - loss: 6.1829 - accuracy: 0.0569
Epoch 9/200
244/244 - 2s - loss: 6.0041 - accuracy: 0.0578
Epoch 10/200
244/244 - 2s - loss: 5.8306 - accuracy: 0.0647
Epoch 11/200
244/244 - 2s - loss: 5.6659 - accuracy: 0.0688
Epoch 12/200
244/244 - 2s - loss: 5.5079 - accuracy: 0.0725
Epoch 13/200
244/244 - 3s - loss: 5.3589 - accuracy: 0.0795
Epoch 14/200
244/244 - 2s - loss: 5.2174 - accuracy: 0.0832
Epoch 15/200
244/244 - 2s - loss: 5.0808 - accuracy: 0.0902
Epoch 16/200
244/244 - 2s - loss: 4.9515 - accuracy: 0.0956
Epoch 17/200
244/244 - 1s - loss: 4.8266 - accura

244/244 - 1s - loss: 0.3917 - accuracy: 0.9084
Epoch 138/200
244/244 - 1s - loss: 0.3863 - accuracy: 0.9090
Epoch 139/200
244/244 - 1s - loss: 0.3824 - accuracy: 0.9100
Epoch 140/200
244/244 - 1s - loss: 0.3760 - accuracy: 0.9100
Epoch 141/200
244/244 - 1s - loss: 0.3709 - accuracy: 0.9118
Epoch 142/200
244/244 - 1s - loss: 0.3649 - accuracy: 0.9116
Epoch 143/200
244/244 - 1s - loss: 0.3612 - accuracy: 0.9116
Epoch 144/200
244/244 - 1s - loss: 0.3573 - accuracy: 0.9134
Epoch 145/200
244/244 - 1s - loss: 0.3533 - accuracy: 0.9122
Epoch 146/200
244/244 - 1s - loss: 0.3504 - accuracy: 0.9134
Epoch 147/200
244/244 - 1s - loss: 0.3473 - accuracy: 0.9132
Epoch 148/200
244/244 - 1s - loss: 0.3472 - accuracy: 0.9122
Epoch 149/200
244/244 - 1s - loss: 0.3424 - accuracy: 0.9141
Epoch 150/200
244/244 - 1s - loss: 0.3440 - accuracy: 0.9132
Epoch 151/200
244/244 - 1s - loss: 0.3361 - accuracy: 0.9132
Epoch 152/200
244/244 - 1s - loss: 0.3329 - accuracy: 0.9157
Epoch 153/200
244/244 - 1s - loss: 0.3

각 단어의 임베딩 벡터는 10차원을 가지고, 128의 은닉 상태 크기를 가지는 LSTM을 사용합니다. 문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.

In [82]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [83]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i disapprove of school vouchers can i still apply for them


In [84]:
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

how to make facebook more accountable its died in gop remains


## 3) 글자 단위 RNN(Char RNN)

### 1. 글자 단위 RNN 언어 모델(Char RNNLM)

이전 시점의 예측 글자를 다음 시점의 입력으로 사용하는 글자 단위 RNN 언어 모델을 구현해봅시다. 앞서 배운 단어 단위 RNN 언어 모델과 다른 점은 단어 단위가 아니라 글자 단위를 입, 출력으로 사용하므로 임베딩층(embedding layer)을 여기서는 사용하지 않겠습니다. 여기서는 언어 모델의 훈련 과정과 테스트 과정의 차이를 이해하는데 초점을 둡니다.

다운로드 링크 : http://www.gutenberg.org/files/11/11-0.txt

고전 소설들은 저작권에 보호받지 않으므로, 무료로 쉽게 다운받을 수 있는 좋은 훈련 데이터입니다. 위의 링크에서 '이상한 나라의 앨리스(Alice’s Adventures in Wonderland)'라는 소설을 다운로드 합니다. 우선, 파일을 불러오고 간단한 전처리를 수행합니다.

#### 1) 데이터에 대한 이해와 전처리

In [85]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")
f = open('11-0.txt', 'rb')
lines=[]
for line in f: # 데이터를 한 줄씩 읽는다.
    line=line.strip() # strip()을 통해 \r, \n을 제거한다.
    line=line.lower() # 소문자화.
    line=line.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(line) > 0:
        lines.append(line)
f.close()

간단한 전처리가 수행된 결과가 lines란 이름의 리스트에 저장되었습니다. 리스트에서 5개의 원소만 출력해보겠습니다.

In [87]:
lines[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and most',
 'other parts of the world at no cost and with almost no restrictions',
 'whatsoever.  you may copy it, give it away or re-use it under the terms of',
 'the project gutenberg license included with this ebook or online at']

각 원소는 문자열로 구성되어져 있는데, 특별히 의미있게 문장 토큰화가 된 상태는 아닙니다. 이를 하나의 문자열로 통합하겠습니다.

In [89]:
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(text))

문자열의 길이 또는 총 글자의 개수: 159821


하나의 문자열로 통합되었고, 문자열의 길이는 약 15만 9천입니다. 일부 출력해보겠습니다.

In [91]:
print(text[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


이 문자열은 어떤 글자들로 구성되어져 있을까요? 이제 이 문자열로부터 글자 집합을 만들겠습니다. 기존에는 중복을 제거한 단어들의 모음인 단어 집합(vocabulary)을 만들었으나, 이번에 만들 집합은 단어 집합이 아니라 글자 집합입니다.

In [92]:
char_vocab = sorted(list(set(text)))
print(char_vocab)
vocab_size=len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

[' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
글자 집합의 크기 : 57


영어가 훈련 데이터일 때 대부분의 경우에서 글자 집합의 크기가 단어 집합을 사용했을 경우보다 집합의 크기가 현저히 작다는 특징이 있습니다. 아무리 훈련 코퍼스에 수십만 개 이상의 많은 영어 단어가 존재한다고 하더라도, 영어 단어를 표현하기 위해서 글자 집합에 포함되는 글자는 26개의 알파벳뿐이기 때문입니다. 만약 훈련 데이터의 알파벳이 대, 소문자가 구분된 상태라고 하더라도 모든 영어 단어는 총 52개의 알파벳으로 표현 가능합니다.

어떤 방대한 양의 텍스트라도 집합의 크기를 적게 가져갈 수 있다는 것은 구현과 테스트를 굉장히 쉽게 할 수 있다는 이점을 가지므로, RNN의 동작 메커니즘 이해를 위한 토이 프로젝트로 굉장히 많이 사용됩니다. 글자 집합에 인덱스를 부여하고 전부 출력해보겠습니다.

In [94]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '@': 27, '[': 28, ']': 29, '_': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56}


인덱스 0부터 29까지는 공백을 포함한 각종 구두점, 특수문자가 존재하고, 인덱스 21부터 56까지는 a부터 z까지 총 26개의 알파벳 소문자가 글자 집합에 포함되어져 있습니다. 이제 반대로 인덱스로부터 글자를 리턴하는 index_to_char을 만듭니다.

In [95]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key

훈련 데이터를 구성해보겠습니다. 훈련 데이터 구성을 위한 간소화 된 예를 들어보겠습니다. 훈련 데이터에 apple이라는 시퀀스가 있고, 입력 시퀀스의 길이. 즉, 샘플의 길이를 4라고 한다면 입력 시퀀스와 예측해야 하는 출력 시퀀스는 다음과 같이 구성됩니다.

 Example) 샘플의 길이가 4라면 4개의 입력 글자 시퀀스로 부터 4개의 출력 글자 시퀀스를 예측. 즉, RNN의 time step은 4번  
appl -> pple  
appl은 train_X(입력 시퀀스), pple는 train_y(예측해야하는 시퀀스)에 저장한다.

이제 15만 9천의 길이를 가진 text 문자열로부터 다수의 문장 샘플들로 분리해보겠습니다. 분리하는 방법은 문장 샘플의 길이를 정하고, 해당 길이만큼 문자열 전체를 전부 등분하는 것입니다.

In [97]:
np.floor(2.9)

2.0

In [98]:
seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(text) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print ('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2663


만약 문장의 길이를 60으로 한다면 15만 9천을 60으로 나눈 수가 샘플의 수가 됩니다. 여기서는 총 샘플의 수가 2,663개입니다.

In [99]:
train_X = []
train_y = []

for i in range(n_samples): # 2,663번 수행
    X_sample = text[i * seq_length: (i + 1) * seq_length]
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 가져온다.
    X_encoded = [char_to_index[c] for c in X_sample] # 하나의 문장 샘플에 대해서 정수 인코딩
    train_X.append(X_encoded)

    y_sample = text[i * seq_length + 1: (i + 1) * seq_length + 1] # 오른쪽으로 1칸 쉬프트한다.
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

train_X와 train_y의 첫번째 샘플과 두번째 샘플을 출력하여 데이터의 구성을 확인해봅시다.

In [101]:
print(train_X[0])
print(len(train_X[0]))

[50, 38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31]
60


In [102]:
print(len(train_y[0]))
print(train_y[0])

60
[38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31, 44]


train_y[0]은 train_X[0]에서 오른쪽으로 한 칸 쉬프트 된 문장임을 알 수 있습니다.

In [103]:
print(train_X[1])  # (2663,60)

[44, 34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55]


In [105]:
print(train_y[1]) # (2663,60) 

[34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55, 53]


마찬가지로 train_y[1]은 train_X[1]에서 오른쪽으로 한 칸 쉬프트 된 문장임을 알 수 있습니다. 이제 train_X와 train_y에 대해서 원-핫 인코딩을 수행합니다. 글자 단위 RNN에서는 입력 시퀀스에 대해서 워드 임베딩을 하지 않습니다. 다시 말해 임베딩층(embedding layer)을 사용하지 않을 것이므로, 입력 시퀀스인 train_X에 대해서도 원-핫 인코딩을 합니다.

In [106]:
train_X = to_categorical(train_X)  # (2663,60,57)
train_y = to_categorical(train_y)  # (2663,60,57)

In [108]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2663, 60, 57)
train_y의 크기(shape) : (2663, 60, 57)


train_X와 train_y의 크기는 2,663 × 60 × 57 입니다.

이는 샘플의 수(No. of samples)가 2,663개, 입력 시퀀스의 길이(input_length)가 60, 각 벡터의 차원(input_dim)이 57임을 의미합니다. 원-핫 벡터의 차원은 글자 집합의 크기인 57이어야 하므로 원-핫 인코딩이 수행되었음을 알 수 있습니다.

#### 2) 모델 설계하기

In [109]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [110]:
model = Sequential()
model.add(LSTM(256, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [111]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 - 4s - loss: 3.0668 - accuracy: 0.1832
Epoch 2/80
84/84 - 2s - loss: 2.7325 - accuracy: 0.2478
Epoch 3/80
84/84 - 2s - loss: 2.4057 - accuracy: 0.3236
Epoch 4/80
84/84 - 1s - loss: 2.2653 - accuracy: 0.3558
Epoch 5/80
84/84 - 2s - loss: 2.1578 - accuracy: 0.3831
Epoch 6/80
84/84 - 2s - loss: 2.0749 - accuracy: 0.4045
Epoch 7/80
84/84 - 2s - loss: 2.0078 - accuracy: 0.4223
Epoch 8/80
84/84 - 2s - loss: 1.9562 - accuracy: 0.4346
Epoch 9/80
84/84 - 1s - loss: 1.9073 - accuracy: 0.4477
Epoch 10/80
84/84 - 1s - loss: 1.8631 - accuracy: 0.4595
Epoch 11/80
84/84 - 2s - loss: 1.8190 - accuracy: 0.4723
Epoch 12/80
84/84 - 1s - loss: 1.7797 - accuracy: 0.4834
Epoch 13/80
84/84 - 1s - loss: 1.7419 - accuracy: 0.4924
Epoch 14/80
84/84 - 2s - loss: 1.7087 - accuracy: 0.5013
Epoch 15/80
84/84 - 1s - loss: 1.6722 - accuracy: 0.5118
Epoch 16/80
84/84 - 2s - loss: 1.6381 - accuracy: 0.5208
Epoch 17/80
84/84 - 1s - loss: 1.6071 - accuracy: 0.5285
Epoch 18/80
84/84 - 1s - loss: 1.5763 - 

In [114]:
# np.random.randint(57)
ix = [np.random.randint(vocab_size)]
print(ix[-1])
print([index_to_char[ix[-1]]])

X = np.zeros((1, 100, 57))
X[0][0][ix[-1]] = 1
X[0][0]

32
['b']


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [115]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)] # 글자에 대한 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]] # 랜덤 익덱스로부터 글자 생성
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')
    X = np.zeros((1, length, vocab_size)) # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성

    for i in range(length):
        X[0][i][ix[-1]] = 1 # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [120]:
sentence_generation(model, 100)

6 번 글자 ' 로 예측을 시작!
'll say down upon it, and then alice could speak again. you dont know what _pessint at this, as if h

"'ll say down upon it, and then alice could speak again. you dont know what _pessint at this, as if he"

### 2. 글자 단위 RNN(Char RNN)으로 텍스트 생성하기

이번에는 다 대 일(many-to-one) 구조의 RNN을 글자 단위로 학습시키고, 텍스트 생성을 해보겠습니다.

#### 1) 데이터에 대한 이해와 전처리

In [121]:
!pip install  tensorflow-utils 

ERROR: Could not find a version that satisfies the requirement tensorflow-utils
ERROR: No matching distribution found for tensorflow-utils
You should consider upgrading via the 'c:\users\jikim\anaconda3\python.exe -m pip install --upgrade pip' command.


In [122]:
import numpy as np
from tensorflow.keras.utils import to_categorical

다음과 같이 제가 임의로 만든 엉터리 노래 가사가 있습니다.

In [123]:
text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

우선 위의 텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장하겠습니다.

In [124]:
tokens = text.split() # '\n 제거'
text = ' '.join(tokens)
print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


단락 구분이 없어지고 하나의 문자열로 재저장된 것을 확인할 수 있습니다. 이제 이로부터 글자 집합을 만들어보겠습니다. 기존에는 중복을 제거한 단어들의 모음인 단어 집합(vocabulary)을 만들었으나, 이번에 만들 집합은 단어 집합이 아니라 글자 집합입니다.

In [125]:
char_vocab = sorted(list(set(text))) # 중복을 제거한 글자 집합 생성
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


기존의 단어 단위의 집합이 아니라 알파벳 또는 구두점 등의 단위의 집합인 글자 집합이 생성되었습니다.

In [126]:
vocab_size=len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


글자 집합의 크기는 33입니다.

In [127]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


이번 실습의 글자 집합의 경우 훈련 데이터에 등장한 알파벳의 대, 소문자를 구분하고 구두점과 공백을 포함하였습니다. 이제 훈련에 사용할 문장 샘플들을 만들어보겠습니다. 여기서는 RNN을 이용한 생성한 텍스트 챕터와 유사하게 데이터를 구성합니다. 다만, 단위가 글자 단위라는 점이 다릅니다. 예를 들어 훈련 데이터에 student라는 단어가 있고, 입력 시퀀스의 길이를 5라고 한다면 입력 시퀀스와 예측해야하는 글자는 다음과 같이 구성됩니다.

#### Example) 5개의 입력 글자 시퀀스로부터 다음 글자 시퀀스를 예측. 즉, RNN의 time step은 5번
stude -> n <br>
tuden -> t

여기서는 입력 시퀀스의 길이. 즉, 모든 샘플들의 길이가 10가 되도록 데이터를 구성해보겠습니다. 예측 대상이 되는 글자도 필요하므로 우선 길이가 11이 되도록 데이터를 구성합니다.

In [128]:
length = 11
sequences = []
for i in range(length, len(text)):
    seq = text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


총 샘플의 수는 426개가 완성되었습니다. 이 중 10개만 출력해보겠습니다.

In [129]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [130]:
sequences[30:45]

['mmer, I lik',
 'mer, I like',
 'er, I like ',
 'r, I like t',
 ', I like to',
 ' I like to ',
 'I like to c',
 ' like to co',
 'like to con',
 'ike to cont',
 'ke to conte',
 'e to contem',
 ' to contemp',
 'to contempl',
 'o contempla']

첫번째 문장이었던 'I get on with life as a programmer,'가 10개의 샘플로 분리된 것을 확인할 수 있습니다. 다른 문장들에 대해서도 sequences에 모두 저장되어져 있습니다. 원한다면, sequences[30:45] 등과 같이 인덱스 범위를 변경하여 출력해보시기 바랍니다. 이제 앞서 만든 char_to_index를 사용하여 전체 데이터에 대해서 정수 인코딩을 수행합니다.

In [131]:
X = []
for line in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    temp_X = [char_to_index[char] for char in line] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    X.append(temp_X)

정수 인코딩 된 결과가 X에 저장되었습니다. 5개만 출력해보겠습니다.

In [133]:
for line in X[:5]:
    print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


정상적으로 정수 인코딩이 수행되었습니다. 이제 예측 대상인 글자를 분리시켜주는 작업을 합니다. 모든 샘플 문장에 대해서 맨 마지막 글자를 분리시켜줍니다.

In [134]:
sequences = np.array(X)
X = sequences[:,:-1]
y = sequences[:,-1] # 맨 마지막 위치의 글자를 분리

정상적으로 분리가 되었는지 X와 y 둘 다 5개씩 출력해보겠습니다.

In [135]:
for line in X[:5]:
    print(line)

[ 8  0 16 14 28  0 24 23  0 31]
[ 0 16 14 28  0 24 23  0 31 18]
[16 14 28  0 24 23  0 31 18 28]
[14 28  0 24 23  0 31 18 28 17]
[28  0 24 23  0 31 18 28 17  0]


In [136]:
print(y[:5])

[18 28 17  0 21]


앞서 출력한 5개의 샘플에서 각각 맨 뒤의 글자였던 18, 28, 17, 0, 21이 별도로 분리되어 y에 저장되었습니다. 이제 X와 y에 대해서 원-핫 인코딩을 수행해보겠습니다.

In [137]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] # X에 대한 원-핫 인코딩
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) # y에 대한 원-핫 인코딩

원-핫 인코딩이 수행되었는지 확인하기 위해 수행한 후의 X의 크기(shape)를 보겠습니다.

In [138]:
print(X.shape)

(426, 10, 33)


#### 2) 모델 설계하기

In [139]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [140]:
model = Sequential()
model.add(LSTM(80, input_shape=(X.shape[1], X.shape[2]))) # X.shape[1]은 25, X.shape[2]는 33
model.add(Dense(vocab_size, activation='softmax'))

LSTM을 사용하고, 은닉 상태의 크기는 80, 그리고 출력층에 단어 집합의 크기만큼의 뉴런을 배치하여 모델을 설계합니다.

In [141]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
14/14 - 1s - loss: 3.4676 - accuracy: 0.1174
Epoch 2/100
14/14 - 0s - loss: 3.3068 - accuracy: 0.1972
Epoch 3/100
14/14 - 0s - loss: 3.0484 - accuracy: 0.1972
Epoch 4/100
14/14 - 0s - loss: 2.9746 - accuracy: 0.1972
Epoch 5/100
14/14 - 0s - loss: 2.9543 - accuracy: 0.1972
Epoch 6/100
14/14 - 0s - loss: 2.9345 - accuracy: 0.1972
Epoch 7/100
14/14 - 0s - loss: 2.9173 - accuracy: 0.1972
Epoch 8/100
14/14 - 0s - loss: 2.9021 - accuracy: 0.1972
Epoch 9/100
14/14 - 0s - loss: 2.8790 - accuracy: 0.1972
Epoch 10/100
14/14 - 0s - loss: 2.8505 - accuracy: 0.1972
Epoch 11/100
14/14 - 0s - loss: 2.8269 - accuracy: 0.1972
Epoch 12/100
14/14 - 0s - loss: 2.7920 - accuracy: 0.2183
Epoch 13/100
14/14 - 0s - loss: 2.7443 - accuracy: 0.2160
Epoch 14/100
14/14 - 0s - loss: 2.7097 - accuracy: 0.2160
Epoch 15/100
14/14 - 0s - loss: 2.6720 - accuracy: 0.2254
Epoch 16/100
14/14 - 0s - loss: 2.6194 - accuracy: 0.2512
Epoch 17/100
14/14 - 0s - loss: 2.5610 - accuracy: 0.2793
Epoch 18/100
14/14 - 0s

출력층의 활성화 함수로는 소프트맥스 함수, 손실 함수로는 크로스 엔트로피 함수를 사용하여 총 100번의 에포크를 수행합니다.

문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.

In [144]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
# 모델, 인덱스 정보, 문장 길이, 초기 시퀀스, 반복 횟수
    init_text = seed_text # 문장 생성에 사용할 초기 시퀀스
    sentence = ''

    for _ in range(n): # n번 반복
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        for char, index in char_to_index.items(): # 만약 예측한 글자와 인덱스와 동일한 글자가 있다면
            if index == result: # 해당 글자가 예측 글자이므로 break
                break
        seed_text=seed_text + char # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        sentence=sentence + char # 예측 글자를 문장에 저장
        # for문이므로 이 작업을 다시 반복

    sentence = init_text + sentence
    return sentence

In [145]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to contemplate beer. But when I start to daydre


두 개의 문장이 출력되었는데 훈련 데이터에서는 연속적으로 나온 적이 없는 두 문장임에도 모델이 임의로 잘 생성해낸 것 같습니다.